In [ ]:
# | default_exp transforms/resize

# Imports

In [ ]:
# | export


import torch
from monai.data import MetaTensor
from monai.transforms.spatial.array import Resize, TraceKeys

# Transforms

In [ ]:
# | export


class ResizeWithSpacing(Resize):  # Turns out that meta_tensor.pixdim already tracks this and so this class is unhelpful
    def __call__(
        self,
        img: MetaTensor,
        *args,
        **kwargs,
    ) -> MetaTensor:
        old_shape = torch.tensor(img.shape[1:])  # Channel is first dim
        old_spacing = torch.tensor(img.meta["spacing"])
        new_shape = torch.tensor(self.spatial_size)
        new_spacing = old_spacing * old_shape / new_shape
        img.meta["spacing"] = new_spacing

        return super().__call__(img, *args, **kwargs)

    def inverse_transform(self, data: MetaTensor, transform) -> MetaTensor:
        new_shape = data.shape[1:]
        new_spacing = data.meta["spacing"]
        old_shape = transform[TraceKeys.ORIG_SIZE]
        old_spacing = new_spacing * new_shape / old_shape
        data.meta["spacing"] = old_spacing
        return super().inverse_transform(data, transform)

In [ ]:
a = torch.randn(1, 5, 10, 15)
a = MetaTensor(a, meta={"spacing": torch.tensor([2, 2, 2])})
display(a.shape, a.meta)

transform = ResizeWithSpacing(spatial_size=(3, 10, 30))
b = transform(a)
display(b.shape, b.meta)

torch.Size([1, 5, 10, 15])


{
    'spacing': tensor([2, 2, 2]),
    affine: tensor([[1., 0., 0., 0.],
        [0., 1., 0., 0.],
        [0., 0., 1., 0.],
        [0., 0., 0., 1.]], dtype=torch.float64),
    space: RAS
}

/tmp/ipykernel_2844928/2466326269.py:12: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  old_spacing = torch.tensor(img.meta["spacing"])


torch.Size([1, 3, 10, 30])


{
    'spacing': tensor([3.3333, 2.0000, 1.0000]),
    affine: tensor([[ 1.6667,  0.0000,  0.0000,  0.3333],
        [ 0.0000,  1.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.5000, -0.2500],
        [ 0.0000,  0.0000,  0.0000,  1.0000]], dtype=torch.float64),
    space: RAS
}

# nbdev

In [ ]:
!nbdev_export